In [1]:
import ixmp as ix
import message_ix
import pandas as pd

from message_ix_models.util import broadcast, private_data_path
from message_data.model.material.data_methanol_new import broadcast_reduced_df

<IPython.core.display.Javascript object>

Unable to determine R home: [WinError 2] The system cannot find the file specified


In [2]:
mp = ix.Platform("ixmp_dev")

## add SCP tech and demand

In [4]:
model_name = "MESSAGEix-GLOBIOM 1.1-R12-MAGPIE"
scen_name = "baseline_MP76BD1BI00"

scenario = message_ix.Scenario(mp, model_name, scen_name)

sc_clone = scenario.clone(model_name, scen_name+"test_scp")

In [11]:
all_years = scenario.vintage_and_active_years()
years = all_years[all_years["year_act"]==all_years["year_vtg"]]
first_year = 2025 # when should scp be available?? 2025? 2030?
years = years[years["year_act"] >= first_year]

In [3]:
region_setup = "R12"

In [13]:
scp_demand = pd.read_excel(private_data_path("scp", "population_beef_demand.xlsx"),
                           sheet_name=f"demand_{region_setup}")
scp_demand["value"] = scp_demand["value"] / 1000
scp_demand = scp_demand[scp_demand["year"]>=first_year]

input = pd.read_excel(private_data_path("scp", "scp_techno_economic.xlsx"),
                           sheet_name=f"input_{region_setup}")
output = pd.read_excel(private_data_path("scp", "scp_techno_economic.xlsx"),
                           sheet_name=f"output_{region_setup}")

In [14]:
input = input.pipe(broadcast, year_vtg=years["year_vtg"])
input["year_act"] = input["year_vtg"]
output = output.pipe(broadcast, year_vtg=years["year_vtg"])
output["year_act"] = output["year_vtg"]

In [15]:
input = input[input["commodity"]!="NH3"]

In [16]:
df_ccu_rel_act = broadcast_reduced_df(pd.read_excel(private_data_path("scp", "scp_techno_economic.xlsx"),
                                   sheet_name=f"relation_activity_{region_setup}"), "relation_activity")

df_ccu_rel_lo = broadcast_reduced_df(pd.read_excel(private_data_path("scp", "scp_techno_economic.xlsx"),
                                   sheet_name=f"relation_lower_{region_setup}"), "relation_lower")

In [18]:
df = sc_clone.par("land_output", {"commodity": "Food Demand|Microbial protein"})
df["level"] = "final_material"

In [19]:
#sc_clone.check_out()

sc_clone.add_set("technology", "syn_micr_beef")
sc_clone.add_set("commodity", "Food Demand|Microbial protein")
sc_clone.add_set("mode", "hydrogen")

if region_setup == "R11":
    sc_clone.add_set("level", "demand")
    sc_clone.add_set("relation", "CO2_PtX_trans_disp_split") # add relation to mimic CO2 input requirements for SCP
    input = input[input["commodity"]!="NH3"] # remove NH3 input since it is not in R11 base model

sc_clone.add_par("input", input)
sc_clone.add_set("level", "final_material")
sc_clone.add_par("output", output)
sc_clone.add_par("land_input", df)
sc_clone.add_set("relation", "CO2_PtX_trans_disp_split")
sc_clone.add_par("relation_activity", df_ccu_rel_act)
sc_clone.add_par("relation_lower", df_ccu_rel_lo)

In [20]:
sc_clone.commit("add scp model")

In [23]:
#sc_clone.remove_solution()
sc_clone.solve(model="MESSAGE-MACRO")

## build 1000f/600f scenario

In [24]:
scen = sc_clone

In [25]:
mode = "clone"
#mode = "load"

#budget = "1000f"
budget = "600f"

In [26]:
scen.scenario.replace("baseline",budget)

'600f_MP76BD1BI00test_scp'

In [27]:
if mode == "clone":
    scen_budget = scen.clone(model=scen.model, scenario=scen.scenario.replace("baseline",budget), shift_first_model_year=2025)
else:
    scen_budget = message_ix.Scenario(mp, model="ENGAGE-MAGPIE_SSP2_v4.1.8.3_T4.5_r3.1", scenario=budget)

Override keep_solution=True for shift_first_model_year


In [ ]:
if mode == "clone":
    scen_budget = scen.clone(model="ENGAGE-MAGPIE_SSP2_v4.1.8.3_T4.5_r3.1", scenario=budget, keep_solution=False,
            shift_first_model_year=2025)
else:
    scen_budget = message_ix.Scenario(mp, model="ENGAGE-MAGPIE_SSP2_v4.1.8.3_T4.5_r3.1", scenario=budget)

In [28]:
extra = scen_budget.set("cat_year", {"type_year": "cumulative", "year": [2020, 2015, 2010]})
scen_budget.check_out()
scen_budget.remove_set("cat_year", extra)
scen_budget.commit("remove cumulative years from cat_year set")
scen_budget.set("cat_year", {"type_year": "cumulative"})

,type_year,year
0,cumulative,2025
1,cumulative,2030
2,cumulative,2035
3,cumulative,2040
4,cumulative,2045
5,cumulative,2050
6,cumulative,2055
7,cumulative,2060
8,cumulative,2070
9,cumulative,2080


In [29]:
emission_dict = {
            "node": "World",
            "type_emission": "TCE",
            "type_tec": "all",
            "type_year": "cumulative",
            "unit": "???",
        }
if budget == "1000f":
    value = 4046
if budget == "600f":
    value = 2605

df = message_ix.make_df(
    "bound_emission", value=value, **emission_dict
)
scen_budget.check_out()
scen_budget.add_par("bound_emission", df)
scen_budget.commit("add emission bound")
scen_budget.par("bound_emission")

,node,type_emission,type_tec,type_year,value,unit
0,World,TCE,all,cumulative,2605.0,???


In [31]:
scen_budget.solve(model="MESSAGE-MACRO")

## reporting

In [3]:
df = mp.scenario_list()

In [12]:
import pyperclip

In [21]:
scen_list = str(df[df["model"]=="MESSAGEix-GLOBIOM 1.1-R12-MAGPIE"].sort_values("cre_date")["scenario"].tolist())

In [22]:
pyperclip.copy(scen_list)

In [3]:
model_name = "MESSAGEix-GLOBIOM 1.1-R12-MAGPIE"

In [3]:
rep_list = ['baseline_MP76BD0BI78test_scp', '600f_MP76BD0BI78test_scp', 'baseline_MP76BD1BI00test_scp', '600f_MP76BD1BI00test_scp']
# 'baseline_MP76BD0BI70' still to build scp and 600f

In [7]:
scen = message_ix.Scenario(mp, model_name, "1000f_MP00BD0BI78_test_calib_macro")
scen.has_solution()

True

In [10]:
from message_data.tools.post_processing.iamc_report_hackathon import report as reporting

In [5]:
for scen_name in rep_list:
    print(scen_name.replace("test_scp", ""))

baseline_MP76BD0BI78
600f_MP76BD0BI78
baseline_MP76BD1BI00
600f_MP76BD1BI00


In [7]:
rep_list[2:]

['baseline_MP76BD1BI00test_scp', '600f_MP76BD1BI00test_scp']

In [8]:
for scen_name in rep_list[2:]:
    scen = message_ix.Scenario(mp, model_name, scen_name)
    reporting(
        mp,
        scen,
        # NB(PNK) this is not an error; .iamc_report_hackathon.report() expects a
        #         string containing "True" or "False" instead of an actual bool.
        "False",
        scen.model,
        scen_name.replace("_test_calib_macro", ""),
        merge_hist=True,
        merge_ts=True,
        #run_config="materials_run_config.yaml",
    )

NameError: name 'rep_list' is not defined

In [12]:
reporting(
    mp,
    scen,
    # NB(PNK) this is not an error; .iamc_report_hackathon.report() expects a
    #         string containing "True" or "False" instead of an actual bool.
    "False",
    scen.model,
    scen.scenario.replace("_test_calib_macro", ""),
    merge_hist=True,
    merge_ts=True,
    #run_config="materials_run_config.yaml",
)

processing Table: Resource|Extraction
processing Table: Resource|Cumulative Extraction
processing Table: Primary Energy
processing Table: Primary Energy (substitution method)
processing Table: Final Energy
processing Table: Secondary Energy|Electricity
processing Table: Secondary Energy|Heat
processing Table: Secondary Energy
processing Table: Secondary Energy|Gases
processing Table: Secondary Energy|Solids
processing Table: Emissions|CO2
processing Table: Carbon Sequestration
processing Table: Emissions|BC
processing Table: Emissions|OC
processing Table: Emissions|CO
processing Table: Emissions|N2O
processing Table: Emissions|CH4
processing Table: Emissions|NH3
processing Table: Emissions|Sulfur
processing Table: Emissions|NOx
processing Table: Emissions|VOC
processing Table: Emissions|HFC
processing Table: Emissions
processing Table: Emissions
processing Table: Agricultural Demand
processing Table: Agricultural Production
processing Table: Fertilizer Use
processing Table: Fertilizer
